In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from fastai.vision.all import *
from PIL import Image
from IPython import display
from tqdm import tqdm
import cv2
import json
import pytesseract

In [3]:
path = Path ('/home/hue/Codes/samandoon/search_twitter/database/media')

In [4]:
files = get_image_files(path)
len(files)

37201

In [5]:
import matplotlib.pyplot as plt

In [6]:
def display_images(cv_image, title=""):    
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(cv_image)
    plt.title(title)
     # display that image
    plt.show()

In [7]:
learner = load_learner("outputs/fastai_resnet50.pkl")

In [8]:
labels = dict()
num_message = 0
num_other = 0

pbar = tqdm(files[:])
for file_path in pbar:
       
    with learner.no_bar():
        label, _, accuracy = learner.predict(file_path)
        

    labels[os.path.basename(file_path)] = (label, torch.max(accuracy).item())
    #display_images(file_path, f"{label}, {torch.max(accuracy)}")
    
    if label == "message":
        num_message += 1
    else:
        num_other += 1
        
    pbar.set_postfix({'num_message': num_message, 'num_other': num_other})
    


100%|█████████████████████████████████████████████████████████████████████| 37201/37201 [1:08:17<00:00,  9.08it/s, num_message=10510, num_other=26691]


In [9]:
with open("fastai_labels.json", "w") as outfile: 
    json.dump(labels, outfile)

In [10]:
# with open('fastai_labels.json', 'r') as fp:
#     data = json.load(fp)

In [13]:
print(f"{num_message/(num_message+num_other)*100:0.2f}%")

28.25%


In [ ]:
for file_name, (label, accuracy) in labels.items():
    if label == "message":
        file_path = str(path) + "/" + file_name
        image=fix_theme(cv2.imread(file_path))
        display_images(image, f" {file_name} - {label}, {accuracy}")
        
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        print(pytesseract.image_to_string(img_rgb, lang='fas'))
#     print(file_name)